In [258]:
import requests
import os
import pandas as pd
from itertools import repeat

UPGRADE_HEIGHT=4707300
HALT_HEIGHT=4713064

API_ENDPOINT="https://api.osmosis.interbloc.org/cosmos/tx/v1beta1/txs?events=tx.height={}"

MSG_EXIT_POOL="/osmosis.gamm.v1beta1.MsgExitPool"
MSG_JOIN_POOL="/osmosis.gamm.v1beta1.MsgJoinPool"

START_HEIGHT = int(os.getenv("START_HEIGHT", UPGRADE_HEIGHT))
END_HEIGHT = int(os.getenv("END_HEIGHT", HALT_HEIGHT))

def call_endpoint(url, headers = {'Accept': 'application/json'}):
    response = requests.get(url, headers=headers)
    return response.json()

### Download all the data

In [259]:
results = []

for h in range(START_HEIGHT, END_HEIGHT):
    print("Processing ", h)
    url = API_ENDPOINT.format(h)
    response_json = call_endpoint(url)

    for tx in response_json["tx_responses"]:
        results.append(tx)

raw_df = pd.DataFrame.from_records(results)
raw_df.to_csv("data/raw_txs.csv")

### Process the data

In [256]:
# Take only the needed columns
df = raw_df[["height","txhash", "code", "timestamp","tx"]]

# Remove unsuccessful transactions
df = df[df["code"] == 0]


In [257]:
# Expand tx
df = df.join(pd.json_normalize(df.tx)[["body.messages"]]).drop(columns=["tx"])
df = df.explode(column="body.messages")
df.head()


,height,txhash,code,timestamp,body.messages
1,4713062,978C71C25130DB96687FFC19136B20EEBC3C706DF8543F...,0,2022-06-08T02:49:23Z,"{'@type': '/osmosis.gamm.v1beta1.MsgExitPool',..."
2,4713062,098895F05128B63E44E52D6F5A70C04527BF8D4A409754...,0,2022-06-08T02:49:23Z,{'@type': '/ibc.core.client.v1.MsgUpdateClient...
2,4713062,098895F05128B63E44E52D6F5A70C04527BF8D4A409754...,0,2022-06-08T02:49:23Z,{'@type': '/ibc.core.channel.v1.MsgAcknowledge...
3,4713062,F7E3867BEA23AE8589CDFED5A7306817C4713A209EF7C1...,0,2022-06-08T02:49:23Z,"{'@type': '/osmosis.gamm.v1beta1.MsgExitPool',..."
4,4713062,412E31BF4DAFD475DD42E253209AC2A9566E979ED08DE1...,0,2022-06-08T02:49:23Z,{'@type': '/osmosis.gamm.v1beta1.MsgSwapExactA...


In [248]:
df = df.join(pd.json_normalize(df["body.messages"])[["@type", "poolId","shareInAmount","shareOutAmount", "sender", "tokenInMaxs", "tokenOutMins"]])
df = df.drop(columns=["body.messages"])

# Filter messages
df = df[(df["@type"] == MSG_EXIT_POOL) | (df["@type"] == MSG_JOIN_POOL )]
df.head()


,height,txhash,code,timestamp,@type,poolId,shareInAmount,shareOutAmount,sender,tokenInMaxs,tokenOutMins
3,4713062,F7E3867BEA23AE8589CDFED5A7306817C4713A209EF7C1...,0,2022-06-08T02:49:23Z,/osmosis.gamm.v1beta1.MsgExitPool,1,17932307203615818721,NaN,osmo1ea6yv55jzerkeqqz9c36t6wyklh8qpqs0da0n6,NaN,[{'denom': 'ibc/27394FB092D2ECCD56123C74F36E4C...
10,4713062,66F5E29D7B8ECBD20FF29BF218AD5CB332126533EE9C60...,0,2022-06-08T02:49:23Z,/osmosis.gamm.v1beta1.MsgJoinPool,1,NaN,3173148729874826303351,osmo1tg70tuzekpd376dpqr68yx5a7r709w6x8jtxha,[{'denom': 'ibc/27394FB092D2ECCD56123C74F36E4C...,NaN
11,4713062,0529B1006913E1903CE8D87122511108B27C286CFB7F6A...,0,2022-06-08T02:49:23Z,/osmosis.gamm.v1beta1.MsgExitPool,561,562208888633134635427167,NaN,osmo1jfxcl8ja3nnfjduqemptknz2j6nk6502zp3rte,NaN,[{'denom': 'ibc/0EF15DF2F02480ADE0BB6E85D9EBB5...
26,4713062,DE53FBFF2C6D45C354B71DAC8598B3719C01A1EB58CDFD...,0,2022-06-08T02:49:23Z,/osmosis.gamm.v1beta1.MsgJoinPool,1,NaN,310929307959001305211,osmo120qgdephqrn4vzf8pan77ekz488ydtxhggzu99,[{'denom': 'ibc/27394FB092D2ECCD56123C74F36E4C...,NaN
28,4713062,F2D9910CC59B85A867C964684EA6D844D19B8AE3FD3AC7...,0,2022-06-08T02:49:23Z,/osmosis.gamm.v1beta1.MsgJoinPool,613,NaN,42158423188958052,osmo1adtxg4klr2z65zqnxwxv66k5wel7mljutaq6mt,[{'denom': 'ibc/E7B35499CFBEB0FF5778127ABA4FB2...,NaN


In [249]:
# Merge shares into one column
df["shares"] = df["shareInAmount"].combine_first(df["shareOutAmount"])
df.drop(columns=["shareInAmount", "shareOutAmount"], inplace=True)

# Merge tokens into one column
df["tokens"] = df["tokenInMaxs"].combine_first(df["tokenOutMins"])
df.drop(columns=["tokenInMaxs","tokenOutMins"], inplace=True)

,height,txhash,code,timestamp,@type,poolId,sender,tokenInMaxs,tokenOutMins,shares
3,4713062,F7E3867BEA23AE8589CDFED5A7306817C4713A209EF7C1...,0,2022-06-08T02:49:23Z,/osmosis.gamm.v1beta1.MsgExitPool,1,osmo1ea6yv55jzerkeqqz9c36t6wyklh8qpqs0da0n6,NaN,[{'denom': 'ibc/27394FB092D2ECCD56123C74F36E4C...,17932307203615818721
10,4713062,66F5E29D7B8ECBD20FF29BF218AD5CB332126533EE9C60...,0,2022-06-08T02:49:23Z,/osmosis.gamm.v1beta1.MsgJoinPool,1,osmo1tg70tuzekpd376dpqr68yx5a7r709w6x8jtxha,[{'denom': 'ibc/27394FB092D2ECCD56123C74F36E4C...,NaN,3173148729874826303351
11,4713062,0529B1006913E1903CE8D87122511108B27C286CFB7F6A...,0,2022-06-08T02:49:23Z,/osmosis.gamm.v1beta1.MsgExitPool,561,osmo1jfxcl8ja3nnfjduqemptknz2j6nk6502zp3rte,NaN,[{'denom': 'ibc/0EF15DF2F02480ADE0BB6E85D9EBB5...,562208888633134635427167
26,4713062,DE53FBFF2C6D45C354B71DAC8598B3719C01A1EB58CDFD...,0,2022-06-08T02:49:23Z,/osmosis.gamm.v1beta1.MsgJoinPool,1,osmo120qgdephqrn4vzf8pan77ekz488ydtxhggzu99,[{'denom': 'ibc/27394FB092D2ECCD56123C74F36E4C...,NaN,310929307959001305211
28,4713062,F2D9910CC59B85A867C964684EA6D844D19B8AE3FD3AC7...,0,2022-06-08T02:49:23Z,/osmosis.gamm.v1beta1.MsgJoinPool,613,osmo1adtxg4klr2z65zqnxwxv66k5wel7mljutaq6mt,[{'denom': 'ibc/E7B35499CFBEB0FF5778127ABA4FB2...,NaN,42158423188958052


In [252]:
# Expand token information
df.reset_index(inplace=True, drop=True)
df = df.join(pd.json_normalize(pd.json_normalize(df["tokens"])[0]).add_prefix("token0_"))
df = df.join(pd.json_normalize(pd.json_normalize(df["tokens"])[1]).add_prefix("token1_"))
df.drop(columns=["tokens"], inplace=True)
display(df)

,height,txhash,code,timestamp,@type,poolId,sender,shares,token0_denom,token0_amount,token1_denom,token1_amount
0,4713062,F7E3867BEA23AE8589CDFED5A7306817C4713A209EF7C1...,0,2022-06-08T02:49:23Z,/osmosis.gamm.v1beta1.MsgExitPool,1,osmo1ea6yv55jzerkeqqz9c36t6wyklh8qpqs0da0n6,17932307203615818721,ibc/27394FB092D2ECCD56123C74F36E4C1F926001CEAD...,221551,uosmo,1866544
1,4713062,66F5E29D7B8ECBD20FF29BF218AD5CB332126533EE9C60...,0,2022-06-08T02:49:23Z,/osmosis.gamm.v1beta1.MsgJoinPool,1,osmo1tg70tuzekpd376dpqr68yx5a7r709w6x8jtxha,3173148729874826303351,ibc/27394FB092D2ECCD56123C74F36E4C1F926001CEAD...,41214387,uosmo,347225933
2,4713062,0529B1006913E1903CE8D87122511108B27C286CFB7F6A...,0,2022-06-08T02:49:23Z,/osmosis.gamm.v1beta1.MsgExitPool,561,osmo1jfxcl8ja3nnfjduqemptknz2j6nk6502zp3rte,562208888633134635427167,ibc/0EF15DF2F02480ADE0BB6E85D9EBB5DAEA2836D386...,150154777279843,uosmo,6526534362
3,4713062,DE53FBFF2C6D45C354B71DAC8598B3719C01A1EB58CDFD...,0,2022-06-08T02:49:23Z,/osmosis.gamm.v1beta1.MsgJoinPool,1,osmo120qgdephqrn4vzf8pan77ekz488ydtxhggzu99,310929307959001305211,ibc/27394FB092D2ECCD56123C74F36E4C1F926001CEAD...,4038500,uosmo,34023843
4,4713062,F2D9910CC59B85A867C964684EA6D844D19B8AE3FD3AC7...,0,2022-06-08T02:49:23Z,/osmosis.gamm.v1beta1.MsgJoinPool,613,osmo1adtxg4klr2z65zqnxwxv66k5wel7mljutaq6mt,42158423188958052,ibc/E7B35499CFBEB0FF5778127ABA4FB2C4B79A6B8D3D...,1284961,uosmo,44686


## Calculate Excess GAMM

## Count of Join/Exit Transactions

In [253]:
df.groupby('@type').size().reset_index(name='number_of_txs')

,@type,number_of_txs
0,/osmosis.gamm.v1beta1.MsgExitPool,2
1,/osmosis.gamm.v1beta1.MsgJoinPool,3


In [254]:
(df.groupby(['@type', 'sender'])
    .size()
    .reset_index(name='number_of_txs')
    .sort_values(by=['number_of_txs'], ascending=False))

,@type,sender,number_of_txs
0,/osmosis.gamm.v1beta1.MsgExitPool,osmo1ea6yv55jzerkeqqz9c36t6wyklh8qpqs0da0n6,1
1,/osmosis.gamm.v1beta1.MsgExitPool,osmo1jfxcl8ja3nnfjduqemptknz2j6nk6502zp3rte,1
2,/osmosis.gamm.v1beta1.MsgJoinPool,osmo120qgdephqrn4vzf8pan77ekz488ydtxhggzu99,1
3,/osmosis.gamm.v1beta1.MsgJoinPool,osmo1adtxg4klr2z65zqnxwxv66k5wel7mljutaq6mt,1
4,/osmosis.gamm.v1beta1.MsgJoinPool,osmo1tg70tuzekpd376dpqr68yx5a7r709w6x8jtxha,1
